### Cyclic Redundancy Check Code (CRC32)
Let's add and check for simple strands and then run an experiment through Badread

In [1]:
import zlib
import random
from typing import List
import uuid
from utils import read_synthesized_strands_from_file

ModuleNotFoundError: No module named 'utils'

In [ ]:
base_mapping = {'A': '00', 'C': '01', 'G': '10', 'T': '11'}
int_mapping = {'00': 'A', '01': 'C', "10": 'G', "11": 'T'}

In [ ]:

def convert_dna_to_byte_string(strand):
    binary_str = "".join([base_mapping[i] for i in strand])

    padding = (8 - len(binary_str) % 8) % 8
    binary_str = binary_str + "0" * padding  # pad with zeros at the end
    
    byte_string = int(binary_str, 2).to_bytes(
        len(binary_str) // 8, byteorder="big")

    return byte_string

In [ ]:
def generate_random_strand(strand_length: int):
    return "".join([random.choice(
        ['A', 'C', 'T', 'G']) for i in range(strand_length)])

In [ ]:
byte_string = convert_dna_to_byte_string(generate_random_strand(300))

In [ ]:
crc_code = zlib.crc32(byte_string)

In [ ]:
def convert_integer_to_dna(num):
    binary_str = bin(num)[2:].zfill(32)
    if len(binary_str) % 2 != 0:
        binary_str = "0" + binary_str
    return "".join(int_mapping[binary_str[i:i+2]] for i in range(0, len(binary_str), 2))


In [ ]:
# Generate random strand
# Convert to byte string
# Get CRC32
# Convert CRC32 to DNA
# Append to end of DNA

strand_length = 300


strand = generate_random_strand(strand_length)

def get_crc_strand(strand):
    crc = zlib.crc32(convert_dna_to_byte_string(strand))
    dna_crc = convert_integer_to_dna(crc)
    final_strand = strand + dna_crc
    return final_strand

NameError: name 'random' is not defined

In [ ]:
def create_fasta_file(ids: List[str], strands: List[str], output_filepath: str):
    with open(output_filepath, 'w') as f:
        for i, strand in enumerate(strands):
            f.write(f">{ids[i]}\n")
            f.write(strand + '\n\n')

    print(f"File saved as {output_filepath}")

In [ ]:
original_strands = read_synthesized_strands_from_file('d/crc_strands.fasta')[0]

In [ ]:
strands = [get_crc_strand(generate_random_strand(strand_length=200)) for i in range(100)]

In [ ]:
ids = [str(uuid.uuid4()) for i in range(len(strands))]

In [ ]:
create_fasta_file(ids, strands, 'data/crc_strands.fasta')

File saved as data/crc_strands.fasta


### Clustering

In [2]:
%cd ..

c:\Users\Parv\Doc\RA\Projects\clustering_dna_storage


In [3]:
from utils import read_synthesized_strands_from_file

In [36]:
original_strands = read_synthesized_strands_from_file('bird_strands.fasta')[0]

In [37]:
original_strands

['TTAAACAAGGGATTCCGTCCGTCCGAGGCGGTCACCAGGCCCGGACGATCAGTGTACGTTCAGGCCATGCCCGGCTAGGCCGTTCCAGGTACGGCCAGGCGTCCGAGCGCCATGCGAGGCCAAGCCCAGCAGGGAGAGTTCATTCCGGGCATGGAGAGTGCAATCCGTGCTAGGTTAGCCCCTGCCTTGCGAGGTGAGAGCACGCAGTGACGGTCGAGCGCCTGCATTGATGGTGGATCTCCCTCAGAGAGAGTGCAAGCCGTGCTTAGTATGACCAAGTCGCGCTACGAGAGTGCAAACCTTCCTGTGTGAGCCCACGACTTGCGCGGGAAGTGGAATGCTATCGTTGGTAGCCTACCCCGCACGACGGAAGCCTTCCGGCACCCAGTCCTGGCAAGAACAATCCGTGCGAGGTGAGCCCCAGCCTTGCGCGTTCACGTCGGTCATGGTCCGCCTACGTCTGACGAAGGCGGCCTACGTCTGACGAAGGCGGCCTACGTCTGACGAAGGCGGCCTACGTCTGACGAAGGCGGCCTACGTCTGACGAAGGCGGCCTACGTCTGACGAAGGCGGCCTACGTCTGACGAAGGCGGCCTACGTCTGACGAAGGCGGCCTACGTCTGACGAAGGCGGCCTACGTCTGACGAAGGCGCCGGTGGTCAGGCAATGCTAGGCCCTGCCAGGAAAGGCCTGGCAATGACAGGCCAAGACCGGCAAGGGCCGTCCATCGGTCGACAGGCCCACCCAGGCCAGGCTAGGTCAGCCCATGCCAGGCCAGGCCAGGCCAGGCCAGGCCAGGCCAGGCCAGGCCAGGCGAGTCCAGTCCCAGCCGGGAAAGTGCAGGGGTCCCAAGGCCATAACAGTCCAGGCCTGGCAAGGACAGTCCAGGCCAGGCCAGGCCAGGCCAGGCCAGGCCAGGCCAGGCCAGGCCCGGCTAGGGCAGGCCCGGCACCCGGTTCACAGGCCCGGCCAGCAGTCCGGCAACCAGGCGAGGCGAG

In [35]:
original_strands

['strand_TTAAACAAGGGATTCCGTCC',
 'strand_GCCGCTCCGGACCCTTCGAA',
 'strand_CTGAGACACGTTACGATTAT',
 'strand_TTTTAGATATTCAGGCACCC',
 'strand_TCGTGGCAACTTAGGCGCAG',
 'strand_TAAACGGGTTAGTGCGACCT']

In [5]:
import Levenshtein
from clustering import Clustering
from utils import get_fastq_records

In [6]:
records = get_fastq_records('birding.fastq')

252it [00:00, 17670.27it/s]


In [7]:
strands = [str(i.seq) for i in records]
ids = [i.id for i in records]

In [8]:
import matplotlib.pyplot as plt

In [9]:
strand_length = 1129

In [10]:
strands_ = [i for i in strands if len(i) > strand_length - 5 and len(i) < strand_length + 10]

In [11]:
clustering_obj = Clustering(strand_pool=strands_, reference_length=strand_length, n_reference_strands=6, distance_threshold=100)

In [12]:
clustering_obj.run_pipeline(fix_orientation=True)

Clustering strands
Total strands 116


100%|██████████| 116/116 [00:00<00:00, 1391.86it/s]


Number of clusters = 6
Clusters are sorted
Orientation fixed in the strand pool
Generating 106 candidates


100%|██████████| 6/6 [00:15<00:00,  2.65s/it]

Fixing candidate orientations
0.0 candidates are reversed


In [17]:
from strand_reconstruction import make_prediction
from tqdm import tqdm
from utils import reverse_complement
from crc_encoding import get_crc_strand

In [14]:
def validate_crc(strand, info_length=1113):
    return get_crc_strand(strand[:info_length]) == strand or get_crc_strand(strand[:info_length-2]) == strand

In [18]:

def generate_candidates_crc_validated(
        clustered_seqs, n_clusters=200, n_attempts=3,
        strand_length=200, ma_sample_size=10):
    
    validated_strands = []
    for ind, i in enumerate(clustered_seqs[:n_clusters]):  # Iterate through clusters
        # Can be RC remember!
        for k in range(n_attempts): # Repeat n_attempts time
            candidate = make_prediction(i, sample_size=ma_sample_size)  # Make candidate prediction
            rev = reverse_complement(candidate)  # Obtain RC vector
            if validate_crc(candidate):  # Validate CRC code for forward and rc prediction
                validated_strands.append(candidate)
                break
            elif validate_crc(rev):
                validated_strands.append(rev)
                break
            else:
                continue
    
    validated_strands = list(set(validated_strands))
    print(f"{len(validated_strands)} valid strands found")

    return validated_strands

In [19]:
validated_strands = generate_candidates_crc_validated(clustering_obj.clustered_seqs)

6 valid strands found


In [39]:
len(set(validated_strands).intersection(original_strands))

0

In [38]:
validated_strands[0] in original_strands

False

In [42]:
validated_strands

['CTGGTTTGCCTGGTTTATGCAAAAACTGAGTTGAGTTGCACGACACTCGTATTAGTCGATAATGGAAGGGGGATGACCAGCTTTAGCAGACCTCTGGATCTTCCTGGTACGAAATAGCCTGCCGTGAGCTCGCTATACACGATGCCTGATATTTGTGTCCAGCAGTGGTAGGGTTCGGGATTCTTGCATTAGGTTACTTAACATCGTTCTAGTCCGGTTGCTTTCACGTAAGATACCGCGGCATGCGGCCTCATAGTCGCGTTTTACATTTGCTAGAACGTCCGGATTATGTCTAATACAACTAGATATACCAGGCCATCTTGTGGGCCGGGGCTAATGGCATAAAGGGACGCTAGGCACGATGTTGATGTCGGTAAACCTGCACTATGTGGCGCTGCTTCTTTTCTTTTCAATATTAAGGACAAGCCAACGACCTTGAAATATAAAGGCTATCAAAGTGTCCGGAGGTTTCTTACATCGTTTGTCGTCACCTCATAGTCCACTTGTCGTCGCGGAATGCCTCTAGTCTTAATAATCGACCACTTTACAGGCTATAATTAGGCGCCGTAGTTCGAGAGTAAAGGCCAACGGTAGGACATGGTAGTGTACCTGCCAATGTGTGCAGACGGGCAAATATACGTCTGTATAGCCCCGTTGGTAATTATCCGCCGTCCGCCTCTCCCTTAGACTGATGGAATCGTGTTGGGTAGATCACGTAACATACCCTCGTACCCGATACTTGTGGCGTTCTTTTGAATTGGAACATCTCATCTTGCGCATTGACAAAGTCCAGTTTATGCGTATACGGCAACAGCGTGTTACGATTTGTTCGCTGACTTTACCGCAACAATCGGAGGCATTGCGCGATAGAGCGCGATTGACGTCTAAGACACCATTACGGAGGCTAATTGGCTGGGGCCAAATCGCATGGTGTGTAGCGAAAAGAAGGCCGAGAATAGGGGGTATATTGAGTGGAAGGCGGCTGCATTAGCCTTAGC

In [41]:
original_strands

['TTAAACAAGGGATTCCGTCCGTCCGAGGCGGTCACCAGGCCCGGACGATCAGTGTACGTTCAGGCCATGCCCGGCTAGGCCGTTCCAGGTACGGCCAGGCGTCCGAGCGCCATGCGAGGCCAAGCCCAGCAGGGAGAGTTCATTCCGGGCATGGAGAGTGCAATCCGTGCTAGGTTAGCCCCTGCCTTGCGAGGTGAGAGCACGCAGTGACGGTCGAGCGCCTGCATTGATGGTGGATCTCCCTCAGAGAGAGTGCAAGCCGTGCTTAGTATGACCAAGTCGCGCTACGAGAGTGCAAACCTTCCTGTGTGAGCCCACGACTTGCGCGGGAAGTGGAATGCTATCGTTGGTAGCCTACCCCGCACGACGGAAGCCTTCCGGCACCCAGTCCTGGCAAGAACAATCCGTGCGAGGTGAGCCCCAGCCTTGCGCGTTCACGTCGGTCATGGTCCGCCTACGTCTGACGAAGGCGGCCTACGTCTGACGAAGGCGGCCTACGTCTGACGAAGGCGGCCTACGTCTGACGAAGGCGGCCTACGTCTGACGAAGGCGGCCTACGTCTGACGAAGGCGGCCTACGTCTGACGAAGGCGGCCTACGTCTGACGAAGGCGGCCTACGTCTGACGAAGGCGGCCTACGTCTGACGAAGGCGCCGGTGGTCAGGCAATGCTAGGCCCTGCCAGGAAAGGCCTGGCAATGACAGGCCAAGACCGGCAAGGGCCGTCCATCGGTCGACAGGCCCACCCAGGCCAGGCTAGGTCAGCCCATGCCAGGCCAGGCCAGGCCAGGCCAGGCCAGGCCAGGCCAGGCCAGGCGAGTCCAGTCCCAGCCGGGAAAGTGCAGGGGTCCCAAGGCCATAACAGTCCAGGCCTGGCAAGGACAGTCCAGGCCAGGCCAGGCCAGGCCAGGCCAGGCCAGGCCAGGCCAGGCCCGGCTAGGGCAGGCCCGGCACCCGGTTCACAGGCCCGGCCAGCAGTCCGGCAACCAGGCGAGGCGAG

In [30]:
validated_strands

['CTGGTTTGCCTGGTTTATGCAAAAACTGAGTTGAGTTGCACGACACTCGTATTAGTCGATAATGGAAGGGGGATGACCAGCTTTAGCAGACCTCTGGATCTTCCTGGTACGAAATAGCCTGCCGTGAGCTCGCTATACACGATGCCTGATATTTGTGTCCAGCAGTGGTAGGGTTCGGGATTCTTGCATTAGGTTACTTAACATCGTTCTAGTCCGGTTGCTTTCACGTAAGATACCGCGGCATGCGGCCTCATAGTCGCGTTTTACATTTGCTAGAACGTCCGGATTATGTCTAATACAACTAGATATACCAGGCCATCTTGTGGGCCGGGGCTAATGGCATAAAGGGACGCTAGGCACGATGTTGATGTCGGTAAACCTGCACTATGTGGCGCTGCTTCTTTTCTTTTCAATATTAAGGACAAGCCAACGACCTTGAAATATAAAGGCTATCAAAGTGTCCGGAGGTTTCTTACATCGTTTGTCGTCACCTCATAGTCCACTTGTCGTCGCGGAATGCCTCTAGTCTTAATAATCGACCACTTTACAGGCTATAATTAGGCGCCGTAGTTCGAGAGTAAAGGCCAACGGTAGGACATGGTAGTGTACCTGCCAATGTGTGCAGACGGGCAAATATACGTCTGTATAGCCCCGTTGGTAATTATCCGCCGTCCGCCTCTCCCTTAGACTGATGGAATCGTGTTGGGTAGATCACGTAACATACCCTCGTACCCGATACTTGTGGCGTTCTTTTGAATTGGAACATCTCATCTTGCGCATTGACAAAGTCCAGTTTATGCGTATACGGCAACAGCGTGTTACGATTTGTTCGCTGACTTTACCGCAACAATCGGAGGCATTGCGCGATAGAGCGCGATTGACGTCTAAGACACCATTACGGAGGCTAATTGGCTGGGGCCAAATCGCATGGTGTGTAGCGAAAAGAAGGCCGAGAATAGGGGGTATATTGAGTGGAAGGCGGCTGCATTAGCCTTAGC

In [84]:
original_strands_no_crc = [i[:200] for i in original_strands]

In [85]:
candidates = clustering_obj.candidates

In [86]:
validated_strands = []

In [92]:
for i in candidates:
    if get_crc_strand(i[:200]) == i:
        validated_strands.append(i[:200])

In [90]:
len(set(validated_strands))

44

In [ ]:
valid

In [94]:
len(set(original_strands_no_crc).intersection(validated_strands))

44

In [70]:
clustering_obj.clustered_seqs[0]

['TCAGTTACGTATTGCTAAAGATCGCAGCCCATTTGGAGTCGGGTTGTACTCCAACGGGATGGTACTGCCCAAGGTGCGGTTCTCGCGTCATAATCACGTGCCAATTTCCTCCGGACATCCAAAATTTAAATTTTAAGCCGTAACTGCGAGAGCTTTGACGTCGCGCCAGGCCAGCAACGTCATCCTAATCTTACCTACCACATTCTTCAGACATCGCCTTTATTTACCAA',
 'TACTTTGTTCAGTTACGTATTGCTAAAGATCGTCAGCCCATTTGGAGTCGGGTTGTACTCCAACGGGATGGTACTGCCCAAGGTGCGGTTCTCGCGTACATATCACGTGCCAATTTCCTCCGGACATCCAAAATTTAATTTTTAAGCCGTAACTGCGAGAGCTTTGACGTCGCGCCAGGCCAGCAACGTCATCCTAATCTTACCTGCCGCATTCTTCAGGACATCGCCTTTATTTACCAAGC',
 'AAAGATCGTCAGCCCATTTGGAGTCGGGTTGTACTCCAACGGGATGGTACTGCCCAAGGTGCGGTTCTCGCGTACATATCACGTGCCAATTTCCTCCGGACATCCAAAATTTAAATTTTAAGCCGTAACCGCCAGAGCTTTGACGTCGCGCCAGGCCAGCAACGTCATCCTAATCTTACCTATCACATTCTTTAGGACATCGCCTTTATTTACCAA',
 'AAAGATCGTCAGCCCATTTGGAGTCGGGTTGTACTCCAACGGGATGGTACTTGCCCAAGGTGCGGTTCTCGCGTACCATATCACGTGCCAATTTCCTCCGGACATTGAAAATTTAAATTTTAAGCCGTAACTGCGAGAGCTTTGACGTCGCGCCAGGCCAGCAACGTCATCCTAATCTTACCTACCATATTCTTCAGGACATCGCCTTTATTTACCAAAGCAATGC',
 'TATTGCTAAAGATCGTCCAGCCCATTTGGAGTCGGGTTGCTACTCCAACGGGATGGTATGGCCC

In [69]:
clustering_obj.generate_candidates(n_candidates=10, fix_orientation=True)

100%|██████████| 10/10 [00:01<00:00,  5.01it/s]

Fixing candidate orientations
0.0 candidates are reversed


['TTACGTATTGCTAAAGATCGTCAGCCCATTTGGAGTCGGGTTGTACTCCAACGGGATGGTACTGCCCAAGGTGCGGTTCTCGCGTACATATCACGTGCCAATTTCCTCCGGACATCCAAAATTTAAATTTTAAGCCGTAACTGCGAGAGCTTTGACGTCGCGCCAGGCCAGCAACGTCATCCTAATCTTACCTACCACATTCTTCAGGACATCGCCTTTATTTACCAA',
 'AGTTACGTATTGCTTTGCTTATGGATGTTGAGGTCCTTGCGAATATCCACTCTAAAGTTGTTCTCCCCCAATCGCGGAATCTCAAAAACTAGTTGGCCATGCTGAGCCATACCTCACCTACATAATAGGTGACACGTTATCGAACCAGACCCACACTTCACTCATCATTCGTATTAGTGGCACCTTTTTAAGGCGGAGAGACGTAACCCAGCGTTGCCTACGTCACTAAGAGCAATAA',
 'TCTTCAGTTACGTATTGCTATACTTTCATCTCATTAACCGATGTCGCATAGATAGATGAGCTCTGAGAGTGATCGTCAGGTCAATATCGCGGTGGGATGGAGGTTGCATATATATTTAAGGCAAAGCAGATGCACCCCCTGACAAGACTTCAATATAAGGCGATTCTGGCTTTACGAACGCACCCTTTGTTAACACTCAAGTCGGTACATCTAGAGCGGTCCATCCACGATTAAAGCAA',
 'TTCGTTCAGTTACGTATTGCTGGAGCTGTGCTTCTGCTGGCTTCTCATTCTTAGGCCGCTGTCTTGCGCGGAAATGCTGCCATTGAACGGCGAACTCTAGGGTAACAATACTTCGCTGAAGGCCGTGGGGCGCGATTCGGGACTGAAAAGTGCCTACGTCTTCAGAGTCCATCGTTTAATCGATGAAGGCAAGAGAGGTCAACCGGCATTAAAACCGAAGTTCATGCGATGATCTCGGCAATACGTG',
 'GTTCAGTTACGTATTGCTGGACTTCC

In [66]:
clustering_obj.reversed_markers

array([False, False, False, ...,  True, False, False], shape=(2179,))

In [63]:
clustering_obj.clustered_seqs[0]

['TCAGTTACGTATTGCTAAAGATCGCAGCCCATTTGGAGTCGGGTTGTACTCCAACGGGATGGTACTGCCCAAGGTGCGGTTCTCGCGTCATAATCACGTGCCAATTTCCTCCGGACATCCAAAATTTAAATTTTAAGCCGTAACTGCGAGAGCTTTGACGTCGCGCCAGGCCAGCAACGTCATCCTAATCTTACCTACCACATTCTTCAGACATCGCCTTTATTTACCAA',
 'TACTTTGTTCAGTTACGTATTGCTAAAGATCGTCAGCCCATTTGGAGTCGGGTTGTACTCCAACGGGATGGTACTGCCCAAGGTGCGGTTCTCGCGTACATATCACGTGCCAATTTCCTCCGGACATCCAAAATTTAATTTTTAAGCCGTAACTGCGAGAGCTTTGACGTCGCGCCAGGCCAGCAACGTCATCCTAATCTTACCTGCCGCATTCTTCAGGACATCGCCTTTATTTACCAAGC',
 'AAAGATCGTCAGCCCATTTGGAGTCGGGTTGTACTCCAACGGGATGGTACTGCCCAAGGTGCGGTTCTCGCGTACATATCACGTGCCAATTTCCTCCGGACATCCAAAATTTAAATTTTAAGCCGTAACCGCCAGAGCTTTGACGTCGCGCCAGGCCAGCAACGTCATCCTAATCTTACCTATCACATTCTTTAGGACATCGCCTTTATTTACCAA',
 'AAAGATCGTCAGCCCATTTGGAGTCGGGTTGTACTCCAACGGGATGGTACTTGCCCAAGGTGCGGTTCTCGCGTACCATATCACGTGCCAATTTCCTCCGGACATTGAAAATTTAAATTTTAAGCCGTAACTGCGAGAGCTTTGACGTCGCGCCAGGCCAGCAACGTCATCCTAATCTTACCTACCATATTCTTCAGGACATCGCCTTTATTTACCAAAGCAATGC',
 'TATTGCTAAAGATCGTCCAGCCCATTTGGAGTCGGGTTGCTACTCCAACGGGATGGTATGGCCC

In [62]:
len(clustering_obj.candidates[0])

243

In [ ]:
clustering_obj.clustered_seqs

[['...'],
 ['CTTCGTTCAGTTACGTATTGCTGCAAAGATTCTGCTCGG',
  'TTCAGTTACGTATTGCTTGGGCAAGTAGTGTGCCAGGATA',
  'CTTATTCGACCCGAATTGTCCAGCTCCGCTCATTGTGTGTGGTCTAAGATATAGGTGGGCAACATGA',
  'TGCGTGGTCTTGATCCTATATAACGATTTTAAGGTTGTTTCGGCAATACGTAAC',
  'CTGTACTTAGCAATACGTA',
  'GTACTTCGTTCAGTTACGTATTGCTCACCCTCTCCTGTCGCA',
  'CTTCAGTTACGTATTGCTTTATTGCAATGACTTGAAGAGATGTGGAAGCCAGC',
  'TTCGTATTGCTCGAGCGTTGGTGTGAGT',
  'TTCGTTCAGTTTACGTATTGCTTAACGGACCCGT',
  'CGTTCAGTACGTATTGCTGCTGGTGGCAATGCAAATGGTA',
  'TTACGTATTGCTGGGTTCCGCAACAGTGTCCATAGGCACAATTTAAGGTAGTCAGGC',
  'CTAGCTACTAGGCATGAGTGGTGTAACCGCTTGTTCCAGCGTCGGGCGCGGTTGCG',
  'TTCAGTTACGTATTGCTTAGCGC',
  '...'],
 ...]

In [ ]:
clust

In [33]:
clustering_obj.clusters

AttributeError: 'Clustering' object has no attribute 'clusters'